# 🚀 FinSentiment 3.0: Sector Screener (Quarterly Edition)

Versión avanzada para análisis de alta frecuencia (Trimestral) con normalización estratificada.

### Nuevas Características (Fase 9):
1. **Quarterly Pulse**: Ingesta híbrida 10-K (Anual) + 10-Q (Trimestral).
2. **Stratified Z-Score**: Normalización por tipo de documento (corrige el sesgo negativo de los 10-K).
3. **Timeline & Heatmap**: Visualización de la evolución temporal ajustada.
4. **Alpha Hunter QoQ**: Correlación con retornos trimestrales.

In [ ]:
# @title 1. Instalación de Dependencias
!pip install sec-edgar-downloader transformers torch pandas numpy matplotlib seaborn beautifulsoup4 yfinance

In [ ]:
# @title 2. Definición del Motor (Core Engine v2.1 + Quarterly Extension)

import os
import re
import glob
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from collections import Counter
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from transformers import BertTokenizer, BertForSequenceClassification

# Configurar estilos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [14, 7]
plt.rcParams['font.size'] = 12

# --- MÓDULO 1: INGESTA (Quarterly Upgrade) ---
class SECLoader:
    def __init__(self, data_dir="data", email="research@example.com", company="Personal Research"):
        self.data_dir = data_dir
        os.makedirs(os.path.join(data_dir, "raw"), exist_ok=True)
        os.makedirs(os.path.join(data_dir, "processed"), exist_ok=True)
        self.downloader = Downloader(company, email, os.path.join(data_dir, "raw"))

    def download_filings(self, ticker: str, amount: int = 4):
        print(f"[>] ({ticker}) Descargando últimos reportes (Target: {amount}x 10-K, {amount}x 10-Q)...")
        try:
            self.downloader.get("10-K", ticker, limit=amount)
            self.downloader.get("10-Q", ticker, limit=amount)
        except Exception as e:
            print(f"[!] Error en descarga {ticker}: {e}")

    def extract_date(self, content: str) -> str:
        patterns = [
            r'FILED AS OF DATE:\s+(\d{8})',
            r'CONFORMED PERIOD OF REPORT:\s+(\d{8})'
        ]
        for p in patterns:
            match = re.search(p, content)
            if match:
                date_str = match.group(1)
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        return None

    def extract_mda(self, html_content: str) -> str:
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator='\n')
        patterns = [
            r'Item\s+7\.\s+Management', 
            r'Item\s+2\.\s+Management',
            r"Management's\s+Discussion", 
        ]
        start_idx = -1
        for p in patterns:
            match = re.search(p, text, re.IGNORECASE)
            if match: start_idx = match.start(); break
        if start_idx == -1: return text[:50000]
        return text[start_idx:start_idx+30000]

    def process_filings(self, ticker: str):
        base_path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        processed_data = []
        
        for report_type in ["10-K", "10-Q"]:
            type_path = os.path.join(base_path, report_type)
            if not os.path.exists(type_path): continue
            
            for root, _, files in os.walk(type_path):
                for file in files:
                    if file.lower().endswith(".txt") and "primary" not in file:
                        try:
                            with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                                content = f.read()
                            mda = self.extract_mda(content)
                            date = self.extract_date(content)
                            if len(mda) > 500:
                                processed_data.append({
                                    'text': mda, 
                                    'date': date, 
                                    'type': report_type,
                                    'accession': file
                                })
                        except: pass
        
        return sorted(processed_data, key=lambda x: x.get('date', '1900'), reverse=True)

# --- MÓDULO 2 NOISE FILTER ---
class TextPreprocessor:
    def clean_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        return "".join(ch for ch in text if ch.isprintable()).strip()
    
    def is_legal_noise(self, sentence: str) -> bool:
        legal_keywords = [
            'forward-looking', 'safe harbor', 'uncertainty', 'may differ', 
            'subject to error', 'actual results', 'factors that could cause',
            'statements regarding', 'cautionary note', 'risk factors', 
            'include but are not limited to', 'assumptions'
        ]
        return any(kw in sentence.lower() for kw in legal_keywords)

    def split_sentences(self, text):
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', text)
        valid_sentences = []
        for s in sentences:
            s = s.strip()
            if len(s) > 20 and len(s.split()) >= 4:
                if not self.is_legal_noise(s):
                    valid_sentences.append(s)
        return valid_sentences

# --- MÓDULO 3 FINBERT ---
class FinBertModel:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"[*] Cargando FinBERT en {self.device}...")
        self.tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
        self.model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert").to(self.device)
        self.labels = {0: 'positive', 1: 'negative', 2: 'neutral'}

    def predict(self, sentences):
        if not sentences: return pd.DataFrame()
        batch_size = 32
        results = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i+batch_size]
            inputs = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()
            for j, s in enumerate(batch):
                results.append({
                    "sentence": s,
                    "pos_val": probs[j][0],
                    "neg_val": probs[j][1]
                })
        return pd.DataFrame(results)


In [ ]:
# @title 3. Ejecución Batch de Alta Frecuencia (Quarterly)

# Inicializar Modelos
print("[*] Inicializando Motor NLP...")
loader = SECLoader()
model = FinBertModel()
prep = TextPreprocessor()

# --- CONFIGURACIÓN DE USUARIO ---
TICKERS_INPUT = "META, AAPL, MSFT, GOOGL, AMZN" # @param {type:"string"}
NUM_REPORTS = 8 # @param {type:"integer"} 

# Parsear tickers
TICKERS = [t.strip().upper() for t in TICKERS_INPUT.split(',') if t.strip()]
print(f"\n📝 TARGETS: {TICKERS}")
print(f"📑 VENTANA DE ANÁLISIS: {NUM_REPORTS} Reportes (Mix 10-K/10-Q)")

def analyze_quarterly_pulse(ticker_symbol, amount=4):
    print(f"\n" + "-"*50)
    print(f" 🏢 PROCESANDO AGENTE: {ticker_symbol} (Quarterly Mode)")
    print("-"*50)
    try:
        loader.download_filings(ticker_symbol, amount=amount)
        docs = loader.process_filings(ticker_symbol)
        if not docs:
            print(f"[!] No data found for {ticker_symbol}")
            return []
            
        reports_to_scan = docs[:amount]
        # Ordenar CRONOLÓGICAMENTE para los gráficos de linea
        reports_to_scan = sorted(reports_to_scan, key=lambda x: x.get('date', '1900'))

        print(f"[>] Analizando serie temporal de {len(reports_to_scan)} puntos...")
        
        history_results = []
        
        for doc in reports_to_scan:
            report_date = doc.get('date', 'Unknown')
            r_type = doc.get('type', 'UNK')
            print(f"   --> [{r_type}] {report_date}")
            
            sentences = prep.split_sentences(prep.clean_text(doc['text']))
            if len(sentences) < 5: continue
                
            df = model.predict(sentences)
            
            if not df.empty:
                net_score = df['pos_val'].mean() - df['neg_val'].mean()
                history_results.append({
                    'ticker': ticker_symbol,
                    'date': report_date,
                    'type': r_type,
                    'net_score': net_score
                })
        
        return history_results
        
    except Exception as e:
        print(f"[ERROR] {ticker_symbol}: {e}")
        return []

# --- BUCLE PRINCIPAL ---
full_history_data = []

for t in TICKERS:
    results = analyze_quarterly_pulse(t, amount=NUM_REPORTS)
    full_history_data.extend(results)

# Construir DataFrame
df_history = pd.DataFrame(full_history_data)

if not df_history.empty:
    df_history['date_obj'] = pd.to_datetime(df_history['date'])
    
    # --- STRATIFIED NORMALIZATION (New Logic) ---
    print("\n[*] Aplicando Stratified Normalization (Tipo de Reporte)... ")
    
    # 1. Agrupar por 'type' (10-K vs 10-Q) para calcular estadísticas sectoriales
    # Esto nos da la "Baseline" de cada tipo de documento.
    stats_by_type = df_history.groupby('type')['net_score'].agg(['mean', 'std']).reset_index()
    print("\nStats por Tipo de Reporte (Baseline):")
    display(stats_by_type)
    
    # 2. Aplicar Z-Score usando la media/std correspondiente a su tipo
    def apply_stratified_z(row):
        stats = stats_by_type[stats_by_type['type'] == row['type']].iloc[0]
        mu = stats['mean']
        sigma = stats['std']
        if sigma == 0: sigma = 1
        return (row['net_score'] - mu) / sigma

    df_history['z_score_stratified'] = df_history.apply(apply_stratified_z, axis=1)
    
    # 3. Ordenar para visualización
    df_history = df_history.sort_values(['ticker', 'date_obj'])

    # --- VISUALIZACIÓN 1: TIMELINE ESTRATIFICADO ---
    print("\n" + "="*40)
    print(" STRATIFIED SENTIMENT TIMELINE")
    print("="*40)
    
    plt.figure(figsize=(14, 8))
    sns.lineplot(
        data=df_history, 
        x='date_obj', 
        y='z_score_stratified', 
        hue='ticker', 
        marker='o',
        palette='tab10',
        linewidth=2.5
    )
    
    plt.title('Evolución Trimestral Estabilizada (Z-Score Ajustado por Tipo)', fontsize=16, fontweight='bold')
    plt.xlabel('Fecha de Reporte')
    plt.ylabel('Stratified Z-Score (Desviación Típica vs Baseline del Tipo)')
    plt.axhline(0, color='black', linestyle='--', alpha=0.3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # --- VISUALIZACIÓN 2: ALPHA HUNTER (Ultimo punto) ---
    latest_indices = df_history.groupby('ticker')['date_obj'].idxmax()
    df_latest = df_history.loc[latest_indices].copy()
    
    print("\n[*] Actualizando Alpha Hunter (Quarterly Precision)...")
    min_date = df_latest['date_obj'].min() - timedelta(days=5)
    max_date = datetime.now() + timedelta(days=1)
    tickers_list = df_latest['ticker'].tolist()
    
    try:
        market_data = yf.download(tickers_list, start=min_date, end=max_date, progress=False)['Close']
        if len(tickers_list) == 1: market_data = pd.DataFrame({tickers_list[0]: market_data})

        perf_list = []
        for idx, row in df_latest.iterrows():
            t = row['ticker']
            d_start = row['date_obj']
            d_end = d_start + timedelta(days=90) 
            if d_end > datetime.now(): d_end = datetime.now() - timedelta(days=1)
            
            if t in market_data.columns:
                ts = market_data[t].dropna()
                future_prices = ts[ts.index >= d_start]
                past_prices = ts[ts.index <= d_end]
                
                if not future_prices.empty and not past_prices.empty:
                    p_start = future_prices.iloc[0]
                    p_end = past_prices.iloc[-1]
                    perf_list.append((p_end - p_start) / p_start * 100)
                else:
                    perf_list.append(0.0)
            else:
                perf_list.append(0.0)
        df_latest['price_return_3m'] = perf_list
    except: 
        df_latest['price_return_3m'] = 0.0

    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        data=df_latest, x='z_score_stratified', y='price_return_3m', 
        s=200, hue='ticker', palette='tab10', edgecolor='black', alpha=0.9
    )
    for i in range(df_latest.shape[0]):
        plt.text(
            x=df_latest.z_score_stratified.iloc[i]+0.02, 
            y=df_latest.price_return_3m.iloc[i]+0.2, 
            s=df_latest.ticker.iloc[i], 
            fontweight='bold', fontsize=9
        )
    plt.axvline(0, color='gray', linestyle='--', alpha=0.6)
    plt.axhline(0, color='gray', linestyle='--', alpha=0.6)
    plt.title('Alpha Hunter QoQ (Stratified): Sentiment vs 3-Month Return', fontsize=16, fontweight='bold')
    plt.xlabel('Stratified Z-Score (Adjusted by Report Type)', fontsize=12)
    plt.ylabel('3-Month Price Return (%)', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("[!] No data analyzed.")